## Startup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

import os
import pickle

In [2]:
random_state=42
np.random.seed(random_state)

In [3]:
from scripts.params import get_params

params = get_params()

## Data Retrieval

In [4]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

In [25]:
name=f'finaldf_train_{params["tablename"]}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)


In [26]:
df.head()

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,YPFD.BA_rets,...,BBAR.BA_gk_vol,BBAR_rets,BBAR_log_rets,BBAR_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
2013-01-03,0.007552,0.007524,0.000129,0.010616,0.010560,0.000677,-0.012748,-0.012830,0.001228,-0.006863,...,0.000169,-0.005725,-0.005742,0.000960,0.008830,0.008791,0.000014,0.001247,0.001246,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.006303,-0.006323,0.000208,-0.010043,-0.010094,0.000554,0.004936,...,0.000406,-0.019194,-0.019380,0.000635,0.018043,0.017882,0.000133,-0.005727,-0.005743,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,0.002114,0.002112,0.000063,-0.014493,-0.014599,0.000517,0.010805,...,0.000492,0.015655,0.015534,0.000511,-0.002488,-0.002492,0.000048,-0.009769,-0.009817,0.000022
2013-01-08,0.008285,0.008251,0.000082,-0.008439,-0.008474,0.000153,-0.016176,-0.016309,0.001085,0.049563,...,0.000438,-0.015414,-0.015534,0.000642,0.015356,0.015239,0.000064,-0.001117,-0.001118,0.000082
2013-01-09,0.017826,0.017669,0.000273,0.000000,0.000000,0.000000,0.011958,0.011887,0.005238,0.000000,...,0.000000,-0.003914,-0.003922,0.000147,-0.008671,-0.008709,0.001065,0.017245,0.017098,0.000273


In [27]:
tickerlist=params["tickerlist"]

## HMM Training

In [28]:
range_states=range(1,11)
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range_states)
emptydf.fillna(np.inf)
results_dict_df={stock:emptydf for stock in tickerlist}

In [29]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [30]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]

    param_dict={
        "covariance_type" : "diag", 
        "n_iter" : 500,
        "random_state" : random_state
        #no voy a usar startprob_prior por devlog 20-06-23
        }

    for nstate in range_states:
        print(stock, nstate)
        model = hmm.GaussianHMM(n_components= nstate, **param_dict, verbose=False)
        results = model.fit(insample_data)

        # TODO: aca tengo que poner un check de model convergence
        if results.monitor_.converged:
            try:
                results_dict_df[stock].loc[nstate, "AIC"]=model.aic(insample_data)
                results_dict_df[stock].loc[nstate, "BIC"]=model.bic(insample_data)
            except ValueError:
                pass

^MERV 1


Model is not converging.  Current: 18854.600714436052 is not greater than 18855.73073827267. Delta is -1.1300238366166013
Model is not converging.  Current: 18150.450606484752 is not greater than 18151.01274767698. Delta is -0.5621411922293191
Model is not converging.  Current: 18859.18589914981 is not greater than 18864.433488621307. Delta is -5.247589471498941


^MERV 2
^MERV 3
^MERV 4
^MERV 5
^MERV 6
^MERV 7
^MERV 8
^MERV 9


Model is not converging.  Current: 18906.15286542906 is not greater than 18906.406794253573. Delta is -0.253928824513423
Model is not converging.  Current: 18184.342352220032 is not greater than 18184.373920656522. Delta is -0.03156843649048824


^MERV 10
GGAL.BA 1
GGAL.BA 2


...

Model is not converging.  Current: 16293.223092897138 is not greater than 16297.273067091071. Delta is -4.04997419393294


BBAR 8
BBAR 9


Model is not converging.  Current: 17104.214221792274 is not greater than 17108.370578230322. Delta is -4.156356438048533


BBAR 10


Model is not converging.  Current: 17093.564168934616 is not greater than 17093.57013615478. Delta is -0.005967220164166065


In [32]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]
    
    best_aic_nstate=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_nstate=results_dict_df[stock]["BIC"].astype(float).idxmin()

    aic_best_model[stock]=hmm.GaussianHMM(n_components = best_aic_nstate, **param_dict).fit(insample_data)
    bic_best_model[stock]=hmm.GaussianHMM(n_components = best_bic_nstate, **param_dict).fit(insample_data)

Model is not converging.  Current: 18150.45060648472 is not greater than 18151.012747677036. Delta is -0.5621411923166306
Model is not converging.  Current: 18150.450606484752 is not greater than 18151.01274767698. Delta is -0.5621411922293191


Model is not converging.  Current: 17746.98884698709 is not greater than 17749.887170814392. Delta is -2.898323827303102
Model is not converging.  Current: 17746.98884698709 is not greater than 17749.887170814392. Delta is -2.898323827303102
Model is not converging.  Current: 17223.11172558406 is not greater than 17224.078189622487. Delta is -0.9664640384253289
Model is not converging.  Current: 17223.111725584044 is not greater than 17224.07818962253. Delta is -0.9664640384871745
Model is not converging.  Current: 17103.254564682 is not greater than 17103.695503924744. Delta is -0.44093924274420715
Model is not converging.  Current: 17103.254564681993 is not greater than 17103.695503924708. Delta is -0.4409392427151033
Model is not converging.  Current: 16565.770178801045 is not greater than 16567.146166650848. Delta is -1.3759878498021862
Model is not converging.  Current: 16565.770178800347 is not greater than 16567.146166650873. Delta is -1.375987850526144
Model is not converging. 

In [34]:
def generate_samples_residuals(model, insample_data, oos_data):
    """
    
    """
    # como el modelo es memoryless, sólo necesito 1 día de observación para saber en qué estado estoy
    # por lo tanto, en vez de complicarme con dos datasets, puedo agregarle el ultimo día de insample_data al ppio de oos_data
    # pseudocodigo
    oos_data=pd.concat([insample_data[-1:], oos_data])
    del insample_data

    samples=pd.DataFrame(columns=oos_data.columns)
    residuals=pd.DataFrame(columns=oos_data.columns)

    # for i=0
    for i in range(1,
                   len(oos_data.index)):
        prev_obs=oos_data[i-1:i]

        todays_obs = oos_data[i:i+1]
        todays_date = todays_obs.index

        state=model.decode(prev_obs)[1][-1]
        # decode()[0] is the log probability, decode()[1] is the sequence of states, [-1] is the last state
        # since we have added the last datum of insample_data to oos_data, then the 
            # TODO: revisar que tenga sentido decodear solo el ultimo día.
            # La alternativa es agregar diez días de insample al principio y usar un decode con diez dias, 
            # me quedo con el ultimo valor del array que maximiza la log-likelihood de la secuencia entera
            # pero como es memoryless, not sure if it makesense
        
        sample = model.sample(n_samples=1, random_state=random_state, currstate=state)[0]
        # sample()[0] is the array with observations of the sampled variables, sample()[1] is the value of the currstate
        sample = pd.DataFrame(data=sample, columns=oos_data.columns, index=todays_date)

        samples=pd.concat([samples, sample])   
        # sampling given state t-1
        # observar realización en t+i
        residual = todays_obs-sample
        
        residuals=pd.concat([residuals, residual])
    
    return samples, residuals

In [36]:
samples, residuals = generate_samples_residuals(aic_best_model[params["index"]], insample_data=insample_data, oos_data=insample_data)

C:\Users\alfredo.sampron\AppData\Local\Temp\ipykernel_47128\3298275339.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  samples=pd.concat([samples, sample])
C:\Users\alfredo.sampron\AppData\Local\Temp\ipykernel_47128\3298275339.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  residuals=pd.concat([residuals, residual])


In [37]:
residuals

,BBAR_log_rets,BBAR_gk_vol
2013-01-03,-0.019600,0.000750
2013-01-04,-0.033239,0.000425
2013-01-07,0.001676,0.000302
2013-01-08,-0.029393,0.000432
2013-01-09,-0.017780,-0.000062
...,...,...
2023-05-23,0.016412,0.002582
2023-05-24,-0.002456,0.001250
2023-05-29,-0.013859,-0.000210
2023-05-30,0.004160,0.001003


# Guardado de datos

In [38]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_bestmodels.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

Los modelos sirven los residuos NO!
https://github.com/alfsn/regime-switching-hmm/issues/27

In [39]:
with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_aic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, f"""HMM_univ_{params["tablename"]}_bic_residuals.pickle"""), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)

In [5]:
models={}
comps=[2,3,4]
datacols=["log_rets","gk_vol"]

for key in data.keys():
    print(key)
    for comp in comps:
        print(comp)
        # TODO: Change from single modelname to [key][comp] nested dict
        modelname=f"{key}_{comp}_model"
        predictionname=f"{key}_{comp}_prediction"
        
        X = data[key][datacols].values.reshape(-1, len(datacols))
        # bivariate
        # log returns and intraday volatility        
        models[modelname]=hmm.GaussianHMM(n_components = comp, #no voy a usar startprob_prior por devlog 20-06-23
                                          covariance_type = "diag", 
                                          n_iter = 500,
                                          random_state = random_state)
        models[modelname].fit(X)
        models[predictionname]=models[modelname].predict(X)

^MERV
2


Model is not converging.  Current: 18846.005278247954 is not greater than 18847.134444221098. Delta is -1.1291659731432446
Model is not converging.  Current: 18142.20139769384 is not greater than 18142.85021337392. Delta is -0.6488156800805882
Model is not converging.  Current: 18905.499058655805 is not greater than 18910.228067216696. Delta is -4.729008560891089


3
4
GGAL
2


Model is not converging.  Current: 18038.2624343628 is not greater than 18038.278369587973. Delta is -0.015935225172142964


3


...

4
USD
2
3


Model is not converging.  Current: 21535.656358535536 is not greater than 21535.89830316691. Delta is -0.24194463137246203
Model is not converging.  Current: 19946.186981522354 is not greater than 19948.99284249492. Delta is -2.805860972566734


4
USD_^MERV
2
3


Model is not converging.  Current: 20007.553148425937 is not greater than 20007.581922967158. Delta is -0.028774541220627725


4


In [6]:
# Predict the hidden states corresponding to observed X.
for key in data.keys():
    for comp in comps:
        print(">"*30, key)
        model=models[f"{key}_{comp}_model"]
        prediction=models[f"{key}_{comp}_prediction"]
        print("unique states: ", pd.unique(prediction))
        print("\nStart probabilities:")
        print(model.startprob_)
        print("\nTransition matrix:")
        print(model.transmat_)
        print("\nGaussian distribution means:")
        print(model.means_)
        print("\nGaussian distribution covariances:")
        print(model.covars_)
        print()

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ^MERV
unique states:  [0 1]

Start probabilities:
[9.99999992e-01 7.77089851e-09]

Transition matrix:
[[0.99190044 0.00809956]
 [0.64107689 0.35892311]]

Gaussian distribution means:
[[ 0.0023665   0.00062083]
 [-0.03635524  0.0146091 ]]

Gaussian distribution covariances:
[[[4.65243073e-04 0.00000000e+00]
  [0.00000000e+00 4.63173289e-06]]

 [[1.44224149e-02 0.00000000e+00]
  [0.00000000e+00 1.50614206e-03]]]

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ^MERV
unique states:  [1 0 2]

Start probabilities:
[2.64082753e-02 9.73591725e-01 1.62699196e-11]

Transition matrix:
[[1.36233321e-03 9.98614722e-01 2.29446306e-05]
 [9.77992322e-01 1.06337631e-02 1.13739147e-02]
 [2.70638123e-01 2.45724508e-01 4.83637369e-01]]

Gaussian distribution means:
[[ 0.00147874  0.0006487 ]
 [ 0.00320576  0.00061345]
 [-0.03997792  0.01558215]]

Gaussian distribution covariances:
[[[5.03860880e-04 0.00000000e+00]
  [0.00000000e+00 8.81549832e-06]]

 [[4.56882297e-04 0.00000000e+00]
  [0.00

In [7]:
def plot_close_rets_vol(data, key, comp):
    model=models[f"{key}_{comp}_model"]
    prediction=models[f"{key}_{comp}_prediction"]
    states=set(prediction)

    fig=plt.figure(figsize = (20, 20))
    plt.tight_layout()
    plt.title(f"{key} Close, Log returns and intraday Vol\n{comp} states")

    for subplot, var in zip(range(1,4), ["Close", "log_rets", "gk_vol"]):    
        plt.subplot(3,1,subplot)
        for i in set(prediction):
            state = (prediction == i)
            x = data[key].index[state]
            y = data[key][var].iloc[state]
            plt.plot(x, y, '.')
        plt.legend(states, fontsize=16)
        
        plt.grid(True)
        plt.xlabel("datetime", fontsize=16)
        plt.ylabel(var, fontsize=16)
            
    plt.savefig(os.path.join(resultsroute, "graphs", 
                             f"{comp}_states", 
                             f"{key}_model_{comp}.png"))

In [ ]:
for key in data.keys():
    for comp in comps:
        plot_close_rets_vol(data, key, comp)

## Coefficients 

In [9]:
models.keys()

dict_keys(['^MERV_2_model', '^MERV_2_prediction', '^MERV_3_model', '^MERV_3_prediction', '^MERV_4_model', '^MERV_4_prediction', 'GGAL_2_model', 'GGAL_2_prediction', 'GGAL_3_model', 'GGAL_3_prediction', 'GGAL_4_model', 'GGAL_4_prediction', 'GGAL.BA_2_model', 'GGAL.BA_2_prediction', 'GGAL.BA_3_model', 'GGAL.BA_3_prediction', 'GGAL.BA_4_model', 'GGAL.BA_4_prediction', 'YPF_2_model', 'YPF_2_prediction', 'YPF_3_model', 'YPF_3_prediction', 'YPF_4_model', 'YPF_4_prediction', 'YPFD.BA_2_model', 'YPFD.BA_2_prediction', 'YPFD.BA_3_model', 'YPFD.BA_3_prediction', 'YPFD.BA_4_model', 'YPFD.BA_4_prediction', 'EDN_2_model', 'EDN_2_prediction', 'EDN_3_model', 'EDN_3_prediction', 'EDN_4_model', 'EDN_4_prediction', 'EDN.BA_2_model', 'EDN.BA_2_prediction', 'EDN.BA_3_model', 'EDN.BA_3_prediction', 'EDN.BA_4_model', 'EDN.BA_4_prediction', 'BMA_2_model', 'BMA_2_prediction', 'BMA_3_model', 'BMA_3_prediction', 'BMA_4_model', 'BMA_4_prediction', 'BMA.BA_2_model', 'BMA.BA_2_prediction', 'BMA.BA_3_model', 'BMA.B

In [10]:
for model in models.keys()
    print(model)
    print("Matriz de transicion:")
    print(model.transmat_)
    print("Matriz de emisiones:")
    print(model.means_)

SyntaxError: expected ':' (2188482437.py, line 1)

## HMM Selection

Selecting the Number of States in Hidden Markov Models: Pragmatic Solutions Illustrated Using Animal Movement
https://sci-hub.st/10.1007/s13253-017-0283-8